## Constants in Pluto

repo: https://github.com/privacy-scaling-explorations/halo2curves/tree/main/src/pluto_eris , https://github.com/daira/pluto-eris \
paper: https://eprint.iacr.org/2010/354.pdf

Given an integer $n < 2^{448}$, return its 7 limbs where the first limb is the least significant one.\
Each limb is 64 bits long in big endian.

In [132]:
def u_64_little_endian(n):
    str_hex = hex(n)
    str_hex_without_0x = str_hex[2:]
    full_width_str = '0' * (112 - len(str_hex_without_0x)) + str_hex_without_0x
    assert len(full_width_str) == 112

    res = []
    for i in range(7):
        temp = '0x' + full_width_str[112 - 16 * (i + 1) : 112 - 16 * i]
        res.append(temp)
    return res

Convert a given integer into its little endian NAF form.\
Definition of NAF form: https://en.wikipedia.org/wiki/Non-adjacent_form

In [133]:

def naf_representation(n):
    naf = []
    while n > 0:
        if n % 2 == 1:
            # If n is odd, set the current digit as 2 - (n mod 4).
            digit = 2 - (n % 4)
            n -= digit
        else:
            # If n is even, set the current digit as 0.
            digit = 0
        naf.append(digit)
        n //= 2
    return naf

# Example usage:
# decimal_number = 7
# naf = naf_representation(decimal_number)
# print(naf): [-1, 0, 0, 1]

In [134]:
print(naf_representation(7))

[-1, 0, 0, 1]


### The Base Field $\mathbb{F}_p$
seed $U = -1298074214633708060054710657220608$ \
the base field size $p = 36 U^4 + 36 U^3 + 24 U^2 + 6 U + 1$ \
we define the constant $NEG\_U = -U$:

In [135]:
U = -1298074214633708060054710657220608
NEG_U = -U
NEG_U.bit_length()

111

In [136]:
hex(NEG_U)

'0x4000000000001000008780000000'

The constant $- (6U + 2)$

In [137]:
NEG_SIX_U_PLUS_2 = -(6 * U + 2)

In [138]:
hex(NEG_SIX_U_PLUS_2)

'0x18000000000006000032cfffffffe'

$-(6U + 2)$ in little endian the NAF
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L22-L27

In [139]:
print(naf_representation(NEG_SIX_U_PLUS_2))

[0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, -1, 0, 1, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1]


The length of the list used to store NEG_SIX_U_PLUS_2 \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L22


In [140]:
len(naf_representation(NEG_SIX_U_PLUS_2))

114

Due to the existence of $-1$, the length of NAF is bigger than the usual bit length:

In [141]:
NEG_SIX_U_PLUS_2.bit_length()

113

the base field prime $p$ in terms of $U$:

In [142]:
p_int = 36 * U**4 + 36 * U**3 + 24 * U**2 + 6 * U + 1
hex(p_int)

'0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5cda8a6c7be4a7a5fe8fadffd6a2a7e8c30006b9459ffffcd300000001'

In [143]:
p_int

102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913

Check the group order $p-1$ is divisible by $6$ and $4$

In [144]:
(p_int - 1) % 6 == 0

True

In [145]:
(p_int - 1) % 4 == 0

True

The limbs representation of $p$:\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp.rs#L39-L47

In [146]:
u_64_little_endian(p_int)

['0x9ffffcd300000001',
 '0xa2a7e8c30006b945',
 '0xe4a7a5fe8fadffd6',
 '0x443f9a5cda8a6c7b',
 '0xa803ca76f439266f',
 '0x0130e0000d7f70e4',
 '0x2400000000002400']

### The base field $\mathbb{F}_p$

In [147]:
Fp = GF(p_int)
Fp

Finite Field of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913

### The constant $R$ for Montgomery form
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp.rs#L67-L75

In [149]:
u_64_little_endian((2**448) % p_int)

['0xa000163afffffff9',
 '0x8d68a2aaffd0ef18',
 '0xbf6a760a123e0121',
 '0x2242c7760637089c',
 '0x67e576bf526ff2f5',
 '0xf7a9dfffa183e9bf',
 '0x03ffffffffff03ff']

In [150]:
R = Fp(2**448)
R

11356855067116315761326349333718857071609919219953404605758555192204529273465116571178922486933671965396447230942486297326349317046265

### The Scalar Field $\mathbb{F}_q$
the scalar field size $q = 36 U^4 + 36 U^3 + 18 U^2 + 6 U+1$

In [151]:
q_int = 36 * U**4 + 36 * U**3 + 18 * U**2 + 6 * U + 1
hex(q_int)

'0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5c7a8a6c7be4a775fe8e177fd69ca7e85d60050af41ffffcd300000001'

The limbs representation of $q$:\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fq.rs#L39-L47

In [152]:
u_64_little_endian(q_int)

['0x1ffffcd300000001',
 '0x9ca7e85d60050af4',
 '0xe4a775fe8e177fd6',
 '0x443f9a5c7a8a6c7b',
 '0xa803ca76f439266f',
 '0x0130e0000d7f70e4',
 '0x2400000000002400']

In [153]:
Fq = GF(q_int)
Fq

Finite Field of size 102211695604070082112571065507755096754575920209623522239390234855480569854275933742834077002685857629445612735086326265689167708028929

### The field $\mathbb{F}_{p^2}$ 
defined by $\mathbb{F}_{p^2} = \mathbb{F}_p[u] = \mathbb{F}_p[X] / (X^2 + 5)$ where $X^2 + 5$ is irreducible in $\mathbb{F}_p[X]$\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp2.rs#L14-L27 (TODO: comment)

Check $X^2 + 5$ is irreducible:

In [102]:
Rp = PolynomialRing(Fp, "x")
x = Rp.gen()
(x^2 + 5).is_irreducible()

True

Construct $\mathbb{F}_{p^2}$ from $\mathbb{F}_{p}$ in $u$ such that $u^2 + 5 = 0$

In [154]:
Fp2.<u> = Fp.extension(x^2 + 5)
Fp2

Finite Field in u of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913^2

Check the identity $u^2 + 5 = 0$ in $\mathbb{F}_{p^{2}}$

In [104]:
u**2 + 5 == 0

True

### The twisted curve (Triton) defined by: $E': Y^2 = X^3 + (u + 3)$
see the defintion in: https://github.com/daira/pluto-eris \
and justified in the repo: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/curve.rs#L100-L106


### The field $\mathbb{F}_{p^6}$ 
defined by $\mathbb{F}_{p^6} = \mathbb{F}_{p^2}[v] = \mathbb{F}_{p^2}[X] / (X^3 - 57/(u + 3))$ where $X^3 - 57/(u+3)$ is irreducible in $\mathbb{F}_{p^2}[X]$\

Note that the non-cubic residue $57/(u + 3)$ is not arbitrarily chosen. It is closely related to the equation of the curve:
$$E / \mathbb{F}_p: Y^2 = X^3 + 57$$
and the twisted curve :
$$E' / \mathbb{F}_{p^2}: Y^2 = X^3 + (u + 3)$$
More precisely, let $\xi = 57/(u + 3) \in \mathbb{F}_{p^2}$, the isomorphism is defined by
$$\Phi: E' \rightarrow E: \quad (x, y) \mapsto (x \cdot \xi^{1/3}, y \cdot \xi^{1/2})$$
and
$$\Phi^{-1}: E \rightarrow E': \quad (x, y) \mapsto (x / \xi^{1/3}, y / \xi^{1/2})$$
See more details in Section 3 regarding D-type twist in paper: http://indigo.ie/~mscott/twists.pdf

Suppose $\xi = 57/(u + 3) = c_1 \cdot u + c_0$ where $c_0, c_1 \in \mathbb{F}_p$

In [105]:
xi = 57 * (u + 3)**(-1)
xi

21902506200872160452693799751661806447409125759205040479869336040462288535916310410867718211025404905980594113530317090904334254435763*u + 36504177001453600754489666252769677412348542932008400799782226734103814226527184018112863685042341509967656855883861818173890424059624

$\xi^{(p^2 - 1)/2 }$ is order $2$

In [1]:
xi**((p_int ** 2 - 1)/2) == Fp2(-1)

NameError: name 'xi' is not defined

the coordinate $c_0$ of $\xi$ in the basis $\{ 1, u\}$ in $\mathbb{F}_{p^2}$\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp6.rs#L15-L23

In [107]:
u_64_little_endian(36504177001453600754489666252769677412348542932008400799782226734103814226527184018112863685042341509967656855883861818173890424059624)

['0xddb6da4b5b6db6e8',
 '0x833bf7b35b701d98',
 '0x3f6072240ebe2483',
 '0x73cd928ee056022c',
 '0xce4a7f2a7bcb4495',
 '0xdbda9924971b3a9a',
 '0x0cdb6db6db6dc3b6']

the coordinate $c_1$ of $\xi$ in the basis $\{ 1, u\}$ in $\mathbb{F}_{p^2}$\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp6.rs#L25-L33

In [108]:
u_64_little_endian(21902506200872160452693799751661806447409125759205040479869336040462288535916310410867718211025404905980594113530317090904334254435763)

['0xeb6db62d36db6db3',
 '0xb523fb0536dcde8e',
 '0x8c6d1148d5a5491b',
 '0x457b57ef5366ce1a',
 '0x489319197d79f5f3',
 '0xb71cc2492776bcc3',
 '0x07b6db6db6db756d']

the irreducible polynomial in $\mathbb{F}_{p^2}[X]$: $X^3 - \xi$

In [109]:
Rp2 = PolynomialRing(Fp2, "x")
x = Rp2.gen()
(x^3 - xi).is_irreducible()

True

construct the field $\mathbb{F}_{p^6} = \mathbb{F}_{p^2}[X]/(X^3 - \xi) = \mathbb{F}_{p^2}[v]$

In [110]:
Fp6.<v> = Fp2.extension(x^3 - xi)
Fp6

Univariate Quotient Polynomial Ring in v over Finite Field in u of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913^2 with modulus v^3 + 80309189403197921659877265756093290307166794450418481759520898815028391298359804839848300107093151321928845082944495999982558932931150*u + 65707518602616481358081399254985419342227377277615121439608008121386865607748931232603154633076214717941782340590951272713002763307289

In [111]:
Fp6.is_field()

True

check the identity $v^3 - \xi = 0$

In [112]:
v**3 - xi == 0

True

### The field $\mathbb{F}_p^{12}$
$\mathbb{F}_{p^{12}} = \mathbb{F}_{p^6}[w] = \mathbb{F}_{p^6}[X] / (X^2 - v)$ where $X^2 - v$ is irreducible in $\mathbb{F}_{p^6}[X]$ \
We may use the following constructor to construct $\mathbb{F}_{p^{12}}$ as usual, but it turns out too slow:


In [ ]:
# Rp6 = PolynomialRing(Fp6, "x")
# x = Rp6.gen()
# (x^2 - v).is_irreducible()
# Fp12 = Rp6.quotient(x^2 - v)
# Fp12

### The const FROBENIUS_COEFF_FQ6_C1[1] 
$\xi^{(p-1)/3} = c_1 \cdot u + c_0$

In [113]:
xi**((p_int-1)/3)

92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858*u + 51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611

### the coordinate $c_0$

In [114]:
hex(51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611)

'0x120de97f024c55bc3bc0d351f4c70da1e3886170077a50986f93678bc921dcd5041bc4bb14cc42dc52e787634eccc335a001825382850d03'

In [115]:
u_64_little_endian(51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611)

['0xa001825382850d03',
 '0x52e787634eccc335',
 '0x041bc4bb14cc42dc',
 '0x6f93678bc921dcd5',
 '0xe3886170077a5098',
 '0x3bc0d351f4c70da1',
 '0x120de97f024c55bc']

### the coordinate $c_1$

In [116]:
hex(92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858)

'0x2096f3f804d973afd82becc2ef081b76132461908eadbe3da1a7f5502b7091965efa1ddf4658080413be1b7cd3c9ea0e2772fea378a9b322'

In [117]:
u_64_little_endian(92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858)

['0x2772fea378a9b322',
 '0x13be1b7cd3c9ea0e',
 '0x5efa1ddf46580804',
 '0xa1a7f5502b709196',
 '0x132461908eadbe3d',
 '0xd82becc2ef081b76',
 '0x2096f3f804d973af']

### The const FROBENIUS_COEFF_FP6_C1[2] 
$\xi^{(p^2-1)/3)} = c_1 \cdot u + c_0$

In [118]:
xi**((p_int**2-1)/3)

39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334

### Hence $c_1 = 0$ and we compute $c_0$:

In [119]:
u_64_little_endian(39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334)

['0x100004c37ffffffe',
 '0xc8ad8b38dffaf50c',
 '0xc956d01c903d720d',
 '0x50000d7ee0e4a803',
 '0x00000000360001c9',
 '0x0000000000004800',
 '0x0000000000000000']

### The const XI_TO_P_MINUS_1_OVER_2 derived from Frobenius map
$\xi^{(p-1)/2}$

In [120]:
xi**((p_int-1)/2)

65079093581113076137070936836706683307023886770993453415929204320012344240499906559031972253911619563827341548210931723067336455786996*u + 95958235239618370663357236287665721330778695800924622118898462510960785934385719569712082879717746516106642479264739976661558948422166

### the coordinate $c_0$

In [121]:
u_64_little_endian(95958235239618370663357236287665721330778695800924622118898462510960785934385719569712082879717746516106642479264739976661558948422166)

['0x54cf5ad1c0926216',
 '0x186c1f3ce4a46d4e',
 '0x9c23800ce9c9452f',
 '0x50e0d09ff6d6c08b',
 '0x7cf421e4d46f6666',
 '0x678664ba4b6d8343',
 '0x21cc26d5de0f80f4']

### the coordinate $c_1$

In [122]:
u_64_little_endian(65079093581113076137070936836706683307023886770993453415929204320012344240499906559031972253911619563827341548210931723067336455786996)

['0xc0505f4c260e91f4',
 '0xe7bbd15f10723657',
 '0xb4b3e0c35358097e',
 '0x87c56f42a558750d',
 '0x4b7211d23f34f0ae',
 '0xf6839d29e2f0d250',
 '0x16ebe8b2e12a1106']

### final exponentiation

In [123]:
def f_exp_by_x(f, x):
    y = -x
    res = 1
    l = y.bit_length()
    for i in range(l-1, -1, -1):
        res = res**2
        if ((y>>i) & 1) == 1:
            res = res * f
    return 1/res

In [124]:
f_exp_by_x(2, -4)

1/16

### test Miller loop

In [125]:
def test_miller_loop(L):
    l = len(L)
    res = 1
    vec = []
    for i in range(l-2, -1, -1):
        res = res*2
        vec.append(res)
        if L[i] == 1:
            res = res + 1
            vec.append(res)
        elif L[i] == -1:
            res = res - 1
            vec.append(res)
    return res
   
        
        

In [126]:
L = [-1, 0, 0, 1]
test_miller_loop(L)

7

In [127]:
bin(7788445287802248360328263943323646)

'0b11000000000000000000000000000000000000000000000000110000000000000000000110010110011111111111111111111111111111110'

In [128]:
int('0xd201000000010000', 16)

15132376222941642752

In [129]:
15132376222941642752.bit_length()

64